<h1>Chapter 11. Object in the Spirit of Python</h1>

The "Spirit of Python" encompasses Python's principles of simplicity, readability, versatility, and elegance, defining its philosophy and guiding values. When writing a library or framework, programmers anticipate that the classes they utilize will operate in a manner consistent with Python's own provided classes.

<h2>Object Presentation</h2>

In [1]:
import math
from array import array


class Vector2d:
    typecode = 'd'

    # Converting x and y to float type in __init__ method
    # allows to detect errors at an early stage
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    # The presence of the __iter__ method makes Vector2d iterable
    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        # Vector2d is an iterated object,
        # *self supplies the x and y components of the format function
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        # Convert the typecode to bytes and concatenate it with the bytes object
        # obtained through array transformation, constructed by traversing the instance
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        # The modulus of a vector is the length of the hypotenuse
        # of a right triangle with cathetes x and y
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

In [2]:
v1 = Vector2d(3, 4)

`Vector2d` components can be accessed directly as attributes (no read methods)

In [3]:
print(v1.x, v1.y)

3.0 4.0


A `Vector2d` object can be unpacked into a tuple of variables

In [4]:
x, y = v1

`repr` for a `Vector2d` object mimics the source code for constructing an exemplar

In [5]:
x, y

(3.0, 4.0)

In [6]:
v1

Vector2d(3.0, 4.0)

Using `eval` shows that the result of `repr` for `Vector2d` is an exact representation of the constructor call 

In [7]:
v1_clone = eval(repr(v1))

`Vector2d` supports comparison with `==`; this is useful for testing

In [8]:
v1 == v1_clone

True

`print` calls the `str` function, which generates an ordered pair for `Vector2d`

In [9]:
print(v1)

(3.0, 4.0)


`bytes` uses `__bytes__` method to get binary representation

In [10]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

`abs` calls the `__abs__` method to return the modulus of the vector

In [11]:
abs(v1)

5.0

`bool` uses the `__bool__` method to return `False` for a `Vector2d` object of zero length, and `True` otherwise

In [12]:
bool(v1), bool(Vector2d(0, 0))

(True, False)